In [ ]:
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


ModuleNotFoundError: No module named 'transformers'

In [ ]:

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")

# Load model with half-precision
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B",
    torch_dtype=torch.float16,
    device_map="auto"  # Automatically map model to GPU
)

